In [12]:
import math
import os

import polars as pl
import datetime as dt

from estimator.estimator import *
from estimator.estimator.nd import NoiseDistribution

In [13]:
jobs = 16
enable_binary = False
expect_security = 128

In [14]:
data_path = "./security_result.parquet"

schema = {
    "dimension": pl.UInt64,
    "modulus": pl.UInt64,
    "secret_distr": pl.String,
    "secret_stddev": pl.Float64,
    "noise_stddev": pl.Float64,
    "security_bits": pl.Float64,
    "datetime": pl.Datetime,
}

if not os.path.exists(data_path):
    df = pl.DataFrame(None, schema)
    df.write_parquet(data_path)

df = pl.read_parquet(data_path)
print(df.filter(pl.col("security_bits") >= expect_security).head())

shape: (5, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 1024      ┆ 132120577 ┆ Ternary      ┆ 0.707107     ┆ 6.4          ┆ 129.882553   ┆ 2024-11-26   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 17:07:01.099 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 907          │
│ 512       ┆ 2048      ┆ Ternary      ┆ 0.707107     ┆ 0.75         ┆ 128.43

In [15]:
def write_to_data(
    dimension: int,
    modulus: int,
    secret_type: str,
    secret_distr: NoiseDistribution,
    noise_stddev,
    security,
):
    new = pl.DataFrame(
        {
            "dimension": [dimension],
            "modulus": [modulus],
            "secret_distr": [secret_type],
            "secret_stddev": [secret_distr.stddev],
            "noise_stddev": [noise_stddev],
            "security_bits": [security],
            "datetime": [dt.datetime.now()],
        },
        schema,
    )
    print(new)
    df.extend(new)
    df.write_parquet(data_path)

In [16]:
def print_uncheck(
    dimension: int,
    modulus: int,
    secret_type: str,
    secret_distr: NoiseDistribution,
    noise_stddev,
    security,
    pat: str,
):
    uncheck = pl.DataFrame(
        {
            "dimension": [dimension],
            "modulus": [modulus],
            "secret_distr": [secret_type],
            "secret_stddev": [secret_distr.stddev],
            "noise_stddev": [noise_stddev],
            "security_bits": [pat.format(security)],
        },
    )
    print(uncheck)

In [17]:
def check_security(
    dimension: int, modulus: int, secret_distr: NoiseDistribution, noise_stddev
):
    param = LWE.Parameters(
        n=dimension, q=modulus, Xs=secret_distr, Xe=ND.DiscreteGaussian(noise_stddev)
    )
    print(param)

    try:
        result = LWE.estimate(
            param,
            jobs=jobs,
        )
    except Exception as err:
        print("err=", err)
        print("Error Occur!")
    else:
        return min([math.log(res.get("rop", 0), 2) for res in result.values()])

In [18]:
def check_security_with_data(
    dimension: int,
    modulus: int,
    secret_type: str,
    secret_distr: NoiseDistribution,
    noise_stddev,
    force_run_estimate: bool = False,
):
    if force_run_estimate:
        security = check_security(dimension, modulus, secret_distr, noise_stddev)
        write_to_data(
            dimension, modulus, secret_type, secret_distr, noise_stddev, security
        )
        return

    temp = df.filter(dimension=dimension, modulus=modulus, secret_distr=secret_type)
    exact = temp.filter(noise_stddev=noise_stddev)

    if exact.is_empty():
        high = temp.filter(pl.col("security_bits") >= expect_security).sort(
            "security_bits"
        )

        if (not high.is_empty()) and high.item(0, "noise_stddev") <= noise_stddev:
            print_uncheck(
                dimension,
                modulus,
                secret_type,
                secret_distr,
                noise_stddev,
                high.item(0, "security_bits"),
                ">{}",
            )
            return

        low = temp.filter(pl.col("security_bits") < expect_security).sort(
            "security_bits", descending=True
        )

        if (not low.is_empty()) and low.item(0, "noise_stddev") >= noise_stddev:
            print_uncheck(
                dimension,
                modulus,
                secret_type,
                secret_distr,
                noise_stddev,
                low.item(0, "security_bits"),
                "<{}",
            )
            return

        security = check_security(dimension, modulus, secret_distr, noise_stddev)

        write_to_data(
            dimension, modulus, secret_type, secret_distr, noise_stddev, security
        )
    else:
        print(exact)

In [ ]:
# LWE
n1 = 512
q1 = 2048
lwe1_noise_stddev = 0.74

if enable_binary:
    lwe1_key_type = "Binary"
    lwe1_key_distribution = ND.SparseBinary(n1 // 2, n1)
else:
    lwe1_key_type = "Ternary"
    lwe1_key_distribution = ND.SparseTernary(n1 // 4, n1 // 4, n1)

In [ ]:
check_security_with_data(
    n1, q1, lwe1_key_type, lwe1_key_distribution, lwe1_noise_stddev
)

shape: (1, 7)
┌───────────┬─────────┬──────────────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ dimension ┆ modulus ┆ secret_distr ┆ secret_stddev ┆ noise_stddev ┆ security_bits ┆ datetime     │
│ ---       ┆ ---     ┆ ---          ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ u64       ┆ u64     ┆ str          ┆ f64           ┆ f64          ┆ f64           ┆ datetime[μs] │
╞═══════════╪═════════╪══════════════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 512       ┆ 2048    ┆ Ternary      ┆ 0.707107      ┆ 0.74         ┆ 128.154627    ┆ 2024-11-26   │
│           ┆         ┆              ┆               ┆              ┆               ┆ 19:26:25.660 │
│           ┆         ┆              ┆               ┆              ┆               ┆ 842          │
└───────────┴─────────┴──────────────┴───────────────┴──────────────┴───────────────┴──────────────┘


In [ ]:
# BSK 1
N1 = 1024
Q1 = 132120577
rlwe1_stddev = 3.20 * (1 << 1)

if enable_binary:
    rlwe1_key_type = "Binary"
    rlwe1_key_distribution = ND.SparseBinary(N1 // 2, N1)
else:
    rlwe1_key_type = "Ternary"
    rlwe1_key_distribution = ND.SparseTernary(N1 // 4, N1 // 4, N1)

In [ ]:
check_security_with_data(N1, Q1, rlwe1_key_type, rlwe1_key_distribution, rlwe1_stddev)

shape: (1, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 1024      ┆ 132120577 ┆ Ternary      ┆ 0.707107     ┆ 6.4          ┆ 129.882553   ┆ 2024-11-26   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 17:07:01.099 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 907          │
└───────────┴───────────┴──────────────┴──────────────┴──────────────┴───────

In [ ]:
n2 = 1024
ksk_noise_stddev = 3.20 * (1 << 1)
if enable_binary:
    ksk_type = "Binary"
    ksk_distribution = ND.SparseBinary(n2 // 2, n2)
else:
    ksk_type = "Ternary"
    ksk_distribution = ND.SparseTernary(n2 // 4, n2 // 4, n2)

In [ ]:
check_security_with_data(n2, Q1, ksk_type, ksk_distribution, ksk_noise_stddev)

shape: (1, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 1024      ┆ 132120577 ┆ Ternary      ┆ 0.707107     ┆ 6.4          ┆ 129.882553   ┆ 2024-11-26   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 17:07:01.099 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 907          │
└───────────┴───────────┴──────────────┴──────────────┴──────────────┴───────

In [ ]:
N2 = 2048
Q2 = 2251799813640193
rlwe2_nosie_stddev = 3.2

if enable_binary:
    rlwe2_key_type = "Binary"
    rlwe2_key_distribution = ND.SparseBinary(N2 // 2, N2)
else:
    rlwe2_key_type = "Ternary"
    rlwe2_key_distribution = ND.SparseTernary(N2 // 4, N2 // 4, N2)

In [ ]:
check_security_with_data(
    N2, Q2, rlwe2_key_type, rlwe2_key_distribution, rlwe2_nosie_stddev
)

LWEParameters(n=2048, q=2251799813640193, Xs=T(p=512, m=512, n=2048), Xe=D(σ=3.20), m=+Infinity, tag=None)
